In [5]:
import pandas as pd
import numpy as np

df = pd.read_excel('node-edges.xlsx', header=None, )
df.columns = df.iloc[61]
df = df.iloc[0:30]

count = 0
for i in range(0, len(df.columns)):
    if (i % 2 != 0):
        df.columns.values[i] = ("%02d" % count) + ': Destination'
        count += 1
        
slices = []
for i in range(2, len(df.columns), 2):
    s = df.iloc[:, i-2:i]
    s = s.dropna()
    s = s[s.iloc[:,1] != '-'] # edges are marked at entrance AND destination, so missing values can be removed
    slices.append(s)
    
nodes = {}
edges = {}
for s in slices:
    node_name = s.iloc[:,0].name
    node_name = str.strip(node_name.split(':')[1]) # remove number and whitespace from name
    edges.update({e: node_name for e in s.iloc[:,0]})
    nodes.update({node_name: {'node_name': node_name,
                              'transitions': [{'entrance': e, 'destination': d} for e, d in s.iloc],
                              'visited':  False,
                              'previous': {}}})
    
class Graph:
    def __init__(self, nodes, edges):
        self.nodes = nodes
        self.edges = edges
        
graph = Graph(nodes, edges)

In [6]:
for s in slices:
    print(s.iloc[:,0].name)
# print([s.iloc[:,0].name for s in slices])

00: Places of Interest
01: Oldale/Petalburg
02: Rustboro
03: Dewford
04: Slateport/Mauville/Verdanturf
05: Fallarbor/Lavaridge
06: Fortree/Lilycove
07: Mossdeep
08: Pacifidlog
09: Sootopolis
10: Petalburg Woods
11: Rusturf Tunnel
12: Granite Cave
13: Abandoned Ship
14: Jagged Pass
15: Meteor Falls
16: Mirage Tower
17: Route 119/123
18: Mt Pyre
19: Surf Hub
20: Magma Hideout
21: Aqua Hideout
22: Lilycove Contest Hall
23: Lilycove Department Store
24: Seafloor Cavern
25: Pokecenters
26: Victory Road
27: Waterfall Hub
28: E4 Pokecenter
29: Navel Rock
30: Dive Hub
31: Sky Pillar


In [31]:
slices_dict = {s.iloc[:,0].name : s for s in slices}

# TODO: Can this be generalized for dungeons?
def split_pokecenters(pokecenters):
    poke_sub = []
    # current_group = []
    current_prefix = ""
    start = 0
    end = 0
    for row in pokecenters.iloc:
        name = " ".join(row[0].split(" ", 2)[:2])
        if current_prefix == name:
            end += 1
        else:
            poke_sub.append(pokecenters.iloc[start:end])
            start = end
            end += 1
            current_prefix = name
    poke_sub.append(pokecenters.iloc[start:end])
    # The first iteration adds an empty dataframe. This removes it.
    return poke_sub[1:]


def process_pokecenters(key_str, slices_dict):
    poke_sub = split_pokecenters(slices_dict[key_str])
    key_num, key_name = key_str.split(':')
    for i, pc in enumerate(poke_sub):
        # new_key = key_num + '-' + str(i) + ':' + key_name
        new_key = key_str.replace(':', '-%d:' % i)
        pc = pc.rename({key_str: new_key}, axis='columns')
        slices_dict[new_key] = pc

        
process_pokecenters('25: Pokecenters', slices_dict)
slices_dict['25-8: Pokecenters']
list(slices_dict.values())

[61  00: Places of Interest          00: Destination
 0    Rustboro Gym Interior  Pacifidlog Pokecenter W
 2    Mauville Gym Interior            Verdanturf SE
 3   Lavaridge Gym Interior     Seafloor Cavern R3 S
 8                E4 Sidney       Dewford Pokecenter
 10               E4 Glacia      Lilycove Pokecenter
 11                E4 Drake              Fortree NNE
 19                 Groudon   Department Store 3F Up
 20                   Ho-oh              Fortree NNW
 21                   Lugia          Mt Pyre 2F Down
 24                  Regice            Sootopolis W2,
 61 01: Oldale/Petalburg             01: Destination
 3     Oldale Pokecenter  Aqua Hideout B1F S Room NW
 4             Oldale SE            Slateport Museum
 5         Petalburg Gym                Slateport SE
 6        Petalburg Mart            Rusturf Tunnel S
 9          Petalburg SE               Pacifidlog NW
 10         Petalburg SW  Aqua Hideout B1F NE Room N,
 61         02: Rustboro          02: Destin

In [ ]:
import copy

def graph_algo(src, dest, graph):
    if src == dest:
        return True
    src_node = graph.nodes[src]
    src_node['visited'] = True
    queue = [src_node]

    while len(queue) > 0:
        # print('Queue: ', sorted([x['node_name'] for x in queue]))
        current = queue.pop(0)
        # if current['node_name'] == dest:
            # print('Current: %s equals Dest: %s' % (current['node_name'], dest))
            # print('### FOUND IT ####')
            # print(current)
        for t in current['transitions']:
            node_name = graph.edges.get(t['destination'])
            # the node might not exist: Strength, one-way, the 32: Marts thing is not really present in the sheet.
            if node_name:
                node = graph.nodes[node_name]
                if node['visited']:
                    continue
                # add previous
                prev = node['previous']
                if prev.get(current['node_name']):
                    prev[current['node_name']].append(t)
                else:
                    prev[current['node_name']] = [t]
                # mark visited
                # if not node['visited']:
                    # node['visited'] = True
                    # queue.append(node)
                node['visited'] = True
                queue.append(node)
    return graph.nodes[dest]

graph_copy = copy.deepcopy(graph)
dest = graph_algo('Slateport/Mauville/Verdanturf', 'Rustboro', graph_copy)
dest

In [4]:
graph_copy.nodes['Pokecenters']['previous']

NameError: name 'graph_copy' is not defined

In [ ]:
reachable = set()

def reach(node_name, graph, depth_max):
    node = graph.nodes[node_name]
    prev = node['previous'].keys()
    # print(prev)
    for k in prev:
        reachable.add(k)
        if depth_max > 0:
            reach(k, graph, depth_max-1)
reach('Rustboro', graph_copy, 5)
len(reachable)

In [ ]:
name = dest['node_name']
the_node = graph_copy.nodes[name]
prev = the_node['previous']
flat_prev = [[(node[0], trans)] for node in list(prev.items())
    for trans in node[1]]

def list_prev(node_name):
    node = graph_copy.nodes[node_name]
    flat_prev = [(node[0], trans) for node in list(node['previous'].items())
                                    for trans in node[1]]
    
    
flat_prev[0]

In [ ]:
def flatten_prev(node_name, graph):
    node = graph.nodes[node_name]
    return [(node[0], trans) for node in list(node['previous'].items())
                        for trans in node[1]]
def list_prev(node_name, graph):
    node = graph.nodes[node_name]
    flat_prev = flatten_prev(node_name, graph)
    return flat_prev
    

print(flatten_prev('Rustboro', graph_copy)[0])
print(flatten_prev('Pokecenters', graph_copy)[0])
list_prev('Slateport/Mauville/Verdanturf', graph_copy)